In [9]:
import os
import numpy as np
import librosa
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from tensorflow.keras.layers import MaxPooling2D, Flatten, Conv2D, BatchNormalization 
from tensorflow.keras.layers import Lambda, ELU
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import regularizers

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
META_DIR = Path('data/fma_metadata')
AUDIO_DIR = Path('data/fma_small')
CONVERTED_DIR = Path('data/converted/rock_inst')

In [5]:
genre_dict = {'Rock':0, 'Instrumental': 1}

reverse_map = {v: k for k, v in genre_dict.items()}
print(reverse_map)

{0: 'Rock', 1: 'Instrumental'}


In [26]:
npzfile = np.load(CONVERTED_DIR/'train_arr.npz')
X_train = npzfile['X']
y_train = npzfile['y']
# print(X_train.shape, y_train.shape)

['X', 'y']
(5, 640, 128) (5,)


In [27]:
npzfile = np.load(CONVERTED_DIR/'valid_arr.npz')
X_valid = npzfile['X']
y_valid = npzfile['y']
# print(X_valid.shape, y_valid.shape)

['X', 'y']
(1, 640, 128) (1,)


In [30]:
# Sanity check - plot Spectogram

# num = 0
# spectogram = X_train[num]
# genre = np.argmax(y_train[num])
# print(reverse_map[genre])
# plt.figure(figsize=(10, 5))
# librosa.display.specshow(spectogram.T, y_axis='mel', x_axis='time')
# plt.colorbar(format='%+2.0f dB')
# plt.title('Test Melspectogram')
# plt.tight_layout()

In [21]:
# Add dimension for single channel for Conv2D
X_train = X_train[...,None]
X_valid = X_valid[...,None]

In [22]:
X_train.shape

(5, 640, 128, 1)

In [23]:
model = Sequential()

model.add(Conv2D(64, (7, 7), input_shape = (640, 128, 1), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.3))

model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [24]:
history = model.fit(X_train, y_train, batch_size=16, epochs=2,
                    validation_data=(X_valid, y_valid), verbose=1)


Train on 5 samples, validate on 1 samples
Epoch 1/2
5/5 [==============================] - 5s 1s/sample - loss: 1.4774 - accuracy: 0.8000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/2
5/5 [==============================] - 2s 378ms/sample - loss: 434.4389 - accuracy: 0.4000 - val_loss: 41.8413 - val_accuracy: 0.0000e+00
